# Re-create OpenAI's Deep Research using OpenAI Agents SDK

### Four different agents will be made:

1. Searcher Agent

    - Takes search terms from *Planner agent* and then searches online for those terms
    - Uses `WebSearchTool` (one of OpenAI Agents SDK's *hosted tools*)

2. Planner Agent

    - Takes a general question from a user and then forms suitable searches based on it

3. Writer Agent

    - Takes data gathered from searches and then turns it into a summary report

4. Pusher Agent

    - Sends a notification to the user's phone with the summary

In [131]:
# Import packages and modules
import os
import requests
import urllib
import http.client
import asyncio

from dotenv import load_dotenv
from agents import Agent, trace, Runner, WebSearchTool, function_tool
from agents.model_settings import ModelSettings
from pydantic import BaseModel
from IPython.display import display, Markdown
from pprint import pprint

In [132]:
# Initialize Pushover constants
pushover_user = os.getenv("PUSHOVER_USER")
pushover_token = os.getenv("PUSHOVER_TOKEN")
pushover_url = os.getenv("PUSHOVER_URL")

In [133]:
# Load the OpenAI API key
load_dotenv(override=True)

True

## Searcher agent

When given a search term, search the internet for it, and finally return a summary of the results.

In [134]:
# System prompt
INSTRUCTIONS = "You're a research assistant. When given a search term, search the web for that term and \
produce a concise summary of the results. The summary must be 2-3 paragraphs and less than 300 \
words long. Capture the main points. Write succinctly, no need to do complete sentences or good \
grammar. The summary will be consumed by someone synthesizing a report, so it's vital you capture the \
essence and ignore any fluff. Don't include any additional commentary other than the summary itself."

# Create the searcher agent
# Arm it with `WebSearchTool`
searcher_agent = Agent(
    name="SearcherAgent",
    instructions=INSTRUCTIONS,
    tools=[WebSearchTool(search_context_size="low")],
    model="gpt-4.1-mini",
    # Make sure the agent uses the tool
    model_settings=ModelSettings(tool_choice="required"),
)

In [135]:
# User query
message = "What are the most popular and successful AI Agent frameworks in November 2025?"

# Set up tracing
with trace("Search"):
    # Run the searcher agent
    result = await Runner.run(searcher_agent, message)

display(Markdown(result.final_output))

As of November 2025, several AI agent frameworks have gained prominence for their capabilities and adoption:

- **LangChain**: A versatile framework for building context-aware, reasoning applications, offering extensive integrations and reusable components. ([linkedin.com](https://www.linkedin.com/pulse/ai-agent-frameworks-trends-popularity-2025-sachin-tiwari-c5tuc?utm_source=openai))

- **LangGraph**: An extension of LangChain, providing a graph-based orchestration layer for managing long-running, stateful agents with complex branching. ([medium.com](https://medium.com/%40dev-infinity101/top-5-frameworks-for-building-ai-agents-in-2025-ddacc314f19f?utm_source=openai))

- **AutoGen**: Developed by Microsoft, this framework focuses on multi-agent collaboration and asynchronous task execution with human-in-the-loop oversight. ([medium.com](https://medium.com/%40dev-infinity101/top-5-frameworks-for-building-ai-agents-in-2025-ddacc314f19f?utm_source=openai))

- **CrewAI**: A rapidly growing framework emphasizing multi-agent collaboration and workflow design, known for its ease of use and suitability for prototyping. ([medium.com](https://medium.com/%40dev-infinity101/top-5-frameworks-for-building-ai-agents-in-2025-ddacc314f19f?utm_source=openai))

- **Semantic Kernel**: Microsoft's open-source development kit for building enterprise-grade generative AI applications. ([medium.com](https://medium.com/%40dev-infinity101/top-5-frameworks-for-building-ai-agents-in-2025-ddacc314f19f?utm_source=openai))

- **OpenAI Agents SDK**: A lightweight Python framework released in March 2025, focusing on creating multi-agent workflows with comprehensive tracing and guardrails. ([jlcnews.com](https://www.jlcnews.com/post/the-best-ai-agents-in-2025-tools-frameworks-and-platforms-compared?utm_source=openai))

- **Google Agent Development Kit (ADK)**: Announced in April 2025, this modular framework integrates with Google's ecosystem, supporting hierarchical agent compositions. ([jlcnews.com](https://www.jlcnews.com/post/the-best-ai-agents-in-2025-tools-frameworks-and-platforms-compared?utm_source=openai))

- **Manus**: An autonomous AI agent developed by Butterfly Effect Technology, capable of independent reasoning, dynamic planning, and decision-making. ([en.wikipedia.org](https://en.wikipedia.org/wiki/Manus_%28AI_agent%29?utm_source=openai))

- **$Agent^2$**: Introduces an agent-generates-agent framework for reinforcement learning automation, enabling intelligent LLM-driven generation of RL agents. ([arxiv.org](https://arxiv.org/abs/2509.13368?utm_source=openai))

- **Agent Lightning**: A flexible and extensible framework that enables reinforcement learning-based training of large language models for any AI agent. ([arxiv.org](https://arxiv.org/abs/2508.03680?utm_source=openai))

These frameworks are shaping the AI agent landscape, each offering unique features and strengths to cater to diverse application needs. 

### Go look at the trace

https://platform.openai.com/traces

## Planner agent

When given a query, come up with a set of ideas for web searches that could be run.

In [136]:
NUM_SEARCHES = 5

# System prompt
INSTRUCTIONS = f"You're a research assistant. When given a query, come up with a set of web searches \
to perform to best answer the query. Output {NUM_SEARCHES} terms to query for."

# Use Pydantic objects to describe the Schema of the output
# This is sometimes called 'structured outputs'

# A web search
class WebSearchItem(BaseModel):
    # We want the agent to tell us why a search is important,
    # this way we get better outcomes

    # This field forces the agent to operate in a reasoning mode
    # and tell you why it's doing what it's doing
    # before telling you the answer
    # This biases it to be more likely to output tokens
    # consistent with its reasoning
    reason: str
    "Your reasoning for why this search is important to the query"

    # The actual search term
    query: str
    "The search term to use for the web search"

# A list of web searches to perform (a plan)
class WebSearchPlan(BaseModel):
    searches: list[WebSearchItem]
    "A list of web searches to perform to best answer the query"

# Create the planner agent
planner_agent = Agent(
    name="PlannerAgent",
    instructions=INSTRUCTIONS,
    model="gpt-4.1-mini",
    # Passing the plan here ensures the output follows the schema
    output_type=WebSearchPlan,
)

In [137]:
# User query
message = "What are the most popular and successful AI Agent frameworks in November 2025?"

# Set up tracing
with trace("Search"):
    # Run the planner agent
    result = await Runner.run(planner_agent, message)
    pprint(result.final_output)

# Now the agent presents us five queries with reasoning behind them

WebSearchPlan(searches=[WebSearchItem(reason='Identify AI agent frameworks currently considered popular in late 2025', query='most popular AI agent frameworks November 2025'), WebSearchItem(reason='Find metrics or reports on success and adoption of AI agent frameworks', query='successful AI agent platforms 2025 adoption metrics'), WebSearchItem(reason='Look for expert reviews or analysis on AI agent frameworks in 2025', query='top AI agent frameworks expert reviews 2025'), WebSearchItem(reason='Check recent news or articles about leading AI agent technologies in late 2025', query='latest news AI agent frameworks November 2025'), WebSearchItem(reason='Explore developer communities and repositories for trending AI agent frameworks', query='GitHub trending AI agent frameworks 2025')])


## Writer agent

Take the results of the internet searches and write a report.

In [138]:
# System prompt
INSTRUCTIONS = "You're a senior researcher tasked with writing a cohesive report for a research query. \
You'll be provided with the original query, and some initial research done by a research assistant.\
You should first come up with an outline for the report that describes the structure and \
flow of the report. Then, generate the report and return that as your final output.\
The final output should be in markdown format, and it should be lengthy and detailed. Aim \
for 5-10 pages of content, at least 1000 words."

# Use structured outputs again

class ReportData(BaseModel):
    short_summary: str
    "A short 2-3 sentence summary of the findings"

    markdown_report: str
    "The final report"

    follow_up_questions: list[str]
    "Suggested topics to research further"

# Create the writer agent
writer_agent = Agent(
    name="WriterAgent",
    instructions=INSTRUCTIONS,
    model="gpt-4o-mini",
    # Passing the structure here ensures the output follows the schema
    output_type=ReportData,
)

## Pusher agent

Send a push notification to the user's phone with the summary.

In [139]:
# This is called a decorator
# Makes 'push' a function tool instead of a function
@function_tool
# Uses Pushover,
# which can be used to send push notifications to a phone
# https://pushover.net
def push(message: str):
    "Send a push notification with this brief message"
    conn = http.client.HTTPSConnection("api.pushover.net:443")
    conn.request("POST", "/1/messages.json",
        urllib.parse.urlencode({
            "token": pushover_token,
            "user": pushover_user,
            "message": message,
        }), { "Content-type": "application/x-www-form-urlencoded" })
    conn.getresponse()

In [140]:
#push("Hello!")
push

FunctionTool(name='push', description='Send a push notification with this brief message', params_json_schema={'properties': {'message': {'title': 'Message', 'type': 'string'}}, 'required': ['message'], 'title': 'push_args', 'type': 'object', 'additionalProperties': False}, on_invoke_tool=<function function_tool.<locals>._create_function_tool.<locals>._on_invoke_tool at 0x1321c3920>, strict_json_schema=True, is_enabled=True, tool_input_guardrails=None, tool_output_guardrails=None)

In [141]:
# System prompt
INSTRUCTIONS = "You're a member of a research team and will be provided with a short summary of a report. \
When you receive the report summary, you send a push notification to the user using your tool, informing them that the research is complete, \
and including the report summary you receive."

# Create the pusher agent
pusher_agent = Agent(
    name="PusherAgent",
    instructions=INSTRUCTIONS,
    # Equip the agent with the ability to send push notifications
    tools=[push],
    model="gpt-4.1-mini",
    # Make sure the agent uses the tool
    model_settings=ModelSettings(tool_choice="required")
)

## Functions for planning and executing the search, using planner_agent and searcher_agent

In [142]:
async def plan_searches(query: str):
    "Use planner_agent to plan which searches to run for the query"
    print("Planning searches...")
    # Run the planner agent
    result = await Runner.run(planner_agent, f"Query: {query}")
    print(f"Will perform {len(result.final_output.searches)} searches")
    return result.final_output

async def perform_searches(search_plan: WebSearchPlan):
    "Call 'search' once for each item in the search plan"
    print("Searching...")
    tasks = [asyncio.create_task(search(item)) for item in search_plan.searches]
    results = await asyncio.gather(*tasks)
    print("Finished searching")
    return results

async def search(item: WebSearchItem):
    "Use searcher_agent to run a web search for each item in the search plan"
    input = f"Search term: {item.query}\nReason for searching: {item.reason}"
    result = await Runner.run(searcher_agent, input)
    return result.final_output

## Functions for writing the report and sending a push notification, using writer_agent and pusher_agent

In [143]:
async def write_report(query: str, search_results: list[str]):
    "Use writer_agent to write a report based on the search results"
    print("Thinking about report...")
    input = f"Original query: {query}\nSummarized search results: {search_results}"
    result = await Runner.run(writer_agent, input)
    print("Finished writing report")
    return result.final_output

async def send_push(report: ReportData):
    "Use pusher_agent to send a notification to the user"
    print("Pushing...")
    result = await Runner.run(pusher_agent, report.short_summary)
    print("Push sent")
    return report

## Execution

In [144]:
query = "What are the most popular and successful AI Agent frameworks in November 2025"

# Set up tracing
with trace("Research trace"):
    print("Starting research...")
    # Formulate the plan
    search_plan = await plan_searches(query)
    # Do the searches
    search_results = await perform_searches(search_plan)
    # Write the report
    report = await write_report(query, search_results)
    # Send the push notification
    await send_push(report)  
    print("Everything ready!")
display(Markdown(report.markdown_report))

Starting research...
Planning searches...
Will perform 5 searches
Searching...
Finished searching
Thinking about report...
Finished writing report
Pushing...
Push sent
Everything ready!


# AI Agent Frameworks: Popular and Successful Solutions in November 2025

## Table of Contents

1. [Introduction](#introduction)  
2. [Overview of AI Agent Frameworks](#overview-of-ai-agent-frameworks)  
   - 2.1 [LangChain](#langchain)  
   - 2.2 [LangGraph](#langgraph)  
   - 2.3 [AutoGen](#autogen)  
   - 2.4 [CrewAI](#crewai)  
   - 2.5 [Semantic Kernel](#semantic-kernel)  
   - 2.6 [OpenAI Agents SDK](#openai-agents-sdk)  
   - 2.7 [Google Agent Development Kit](#google-agent-development-kit)  
   - 2.8 [Manus](#manus)  
   - 2.9 [Agent Lightning](#agent-lightning)  
   - 2.10 [AutoAgent](#autoagent)  
3. [Adoption Trends and Economic Impact](#adoption-trends-and-economic-impact)  
4. [Challenges and Future Directions](#challenges-and-future-directions)  
5. [Conclusion](#conclusion)  
6. [References](#references)  

## Introduction

As of November 2025, the landscape of artificial intelligence (AI) agent frameworks has evolved significantly, with various frameworks gaining popularity and demonstrating substantial success across diverse industries. This report provides an in-depth analysis of the leading AI agent frameworks, their features, and the operational efficiencies they promise.

## Overview of AI Agent Frameworks

The rise of AI agents is characterized by their capability to automate complex tasks, enhance decision-making, and optimize workflows. Here are some of the most prominent frameworks:

### 2.1 LangChain

LangChain has emerged as a modular framework for building Language Model (LLM)-powered AI agents. Its versatility allows developers to integrate external tools and custom logic. LangChain is particularly known for its applications in financial services and healthcare, making it the most downloaded agent framework globally, as reported by multiple sources.

### 2.2 LangGraph

An extension of LangChain, LangGraph specializes in stateful orchestration for multi-agent systems. It is tailored for complex workflows, offering functionalities such as multi-agent coordination and advanced error handling—a critical asset for adaptive AI applications and scenarios requiring intricate interactions like interactive storytelling.

### 2.3 AutoGen

Developed by Microsoft, AutoGen emphasizes multi-agent collaboration and asynchronous task execution. Its focus on human oversight makes it particularly suitable for enterprise workflows that require high reliability and security. AutoGen’s seamless integration with existing Microsoft ecosystems like Azure AI enhances its appeal.

### 2.4 CrewAI

CrewAI has gained recognition for facilitating collaborative problem-solving environments with speed and simplicity. Its role-based architecture enables diverse expertise to be utilized effectively, making it ideal for scenarios ranging from project management to healthcare coordination.

### 2.5 Semantic Kernel

The Semantic Kernel by Microsoft represents a lightweight framework that bridges traditional app development with AI capabilities. It is designed to enhance enterprise applications, supporting seamless orchestration of AI workflows while ensuring security and compliance.

### 2.6 OpenAI Agents SDK

This lightweight Python framework is aimed at creating multi-agent workflows efficiently. The OpenAI Agents SDK emphasizes detailed tracing and guardrails, ensuring compatibility with over 100 LLMs, thus making it a practical choice for developers.

### 2.7 Google Agent Development Kit

Google's ADK integrates well with its ecosystem, supporting the development of hierarchical agent compositions. It enables developers to write minimal code for complex tasks, enhancing productivity and process automation.

### 2.8 Manus

Launched in March 2025, Manus is an autonomous AI agent capable of independent reasoning and dynamic task management. Its cloud-based operation allows it to function continuously, even without user interaction, thus pioneering the realm of intelligent autonomous agents.

### 2.9 Agent Lightning

Agent Lightning introduces a flexible approach to AI agent training using reinforcement learning. It allows for the easy integration of existing agents and focuses on decoupling agent execution from training mechanisms, marking a significant step in agent development methodologies.

### 2.10 AutoAgent

With a promise of zero-code transformation, AutoAgent allows users to create LLM agents through natural language instructions. Its framework consists of self-managing tools and modules designed for user-friendliness, making it accessible to non-developers.

## Adoption Trends and Economic Impact

The proliferation of AI frameworks has catalyzed an observable shift in organizational strategies, with approximately 79% of businesses indicating some level of AI agent implementation as of November 2025. While only 19% have scaled these solutions enterprise-wide, the economic potential is significant—with predictions of generating up to USD 450 billion by 2028 through enhanced operational efficiencies and cost savings.

Despite this, challenges pertaining to trust and technology integration hinder faster adoption rates. Recent statistics reveal that only 2% of businesses have fully deployed AI agents, with many acknowledging the competitive edge that enhanced deployment could offer.

Operationally, organizations utilizing AI agents report over a 60% reduction in manual labor for repetitive processes, yielding cost efficiencies. Notably, success rates for AI agent deployment have improved to 65%, up from 35% in 2023.

## Challenges and Future Directions

Despite the progress, several challenges remain:
- **Integration Issues**: Many organizations face hurdles in integrating AI systems with legacy systems, which can stall implementation.  
- **Data Privacy**: Concerns surrounding data security and user privacy can impede trust in AI solutions.  
- **Human Oversight**: The need for human oversight remains critical, particularly in sectors where ethical considerations are paramount, such as healthcare and finance.

To mitigate these challenges, future AI frameworks should prioritize user-friendly designs, enhanced security features, and robust mechanisms for human oversight and interaction. The continued evolution of frameworks like LangChain, AutoGen, and others will likely shape the overarching trajectory of AI agent development.

## Conclusion

The emergence of various AI agent frameworks as of November 2025 indicates a robust market poised for transformative change. Frameworks like LangChain, Manus, and AutoGen, among others, are not only redefining operational efficiencies but also paving the way for more sophisticated AI applications. Nonetheless, addressing integration, privacy, and oversight challenges is crucial for realizing the full potential of AI agents in various industries.

## References
1. AppyPie Agents. (2025). Best AI Agent Frameworks. [appypieagents.ai](https://www.appypieagents.ai/blog/best-ai-agent-frameworks?utm_source=openai).
2. Medium. Various Articles on AI Agents. [medium.com](https://medium.com).
3. Wikipedia. Manus (AI Agent). [en.wikipedia.org](https://en.wikipedia.org/wiki/Manus_%28AI_agent%29?utm_source=openai).
4. Capgemini. (2025). AI Agents Economic Impact Report.  
5. TechRadar. (2025). AI Agents Financial Prospects.  
6. Global Risk Community. (2025). AI Agent Market Trends.  
7. 7T.AI. (2025). AI Agent Adoption Statistics.  
8. Various Industry Reports on AI Agents (2025).  

---

### Go look at the trace

https://platform.openai.com/traces